In [1]:
import requests
import hashlib
import time
import json
import time
import whisper
import os
import subprocess
from pathlib import Path
import sys
import tempfile


In [2]:
#api fro podcastindex

API_KEY = "4YCF385ZNXLEHADRUSFV"
API_SECRET = "YcjAfd9x7S$$qX^4u#n$rvLD$X6c^pT87ShWvF3V"

In [3]:
#set up podcastindex api and retrive 50 treending podcast(random engagement/recent top50)

timestamp = str(int(time.time()))
auth_hash = hashlib.sha1((API_KEY + API_SECRET + timestamp).encode('utf-8')).hexdigest()

headers = {
    "X-Auth-Date": timestamp,
    "X-Auth-Key": API_KEY,
    "Authorization": auth_hash,
    "User-Agent": "PodcastIndex-Client"
}

# Step 1: Get Trending Podcasts
url = "https://api.podcastindex.org/api/1.0/podcasts/trending"
params = {"max": 50}  # Get up to 200 trending podcasts

response = requests.get(url, headers=headers, params=params)
trending_podcasts = response.json().get("feeds", [])

print(f"Retrieved {len(trending_podcasts)} trending podcasts.")

Retrieved 50 trending podcasts.


In [4]:
#save top 50 podcast json

url = "https://api.podcastindex.org/api/1.0/podcasts/trending"
response = requests.get(url, headers=headers, params=params)
trending_data = response.json()

#Extract relevant fields- only retrievable meta data from podcstindex
trending_data = response.json()
trending_podcasts = trending_data.get("feeds", [])

simplified_podcasts = [
    {
        "title": feed.get("title"),
        "url": feed.get("url"),
        "description": feed.get("description"),
        "id": feed.get("id"),
        "author": feed.get("author"),
    }
    for feed in trending_podcasts
]


#Save to JSON file
output_path = "top_podcasts.json"
with open(output_path, "w") as f:
    json.dump(simplified_podcasts, f, indent=4)

print(f"Saved {len(simplified_podcasts)} podcasts to {output_path}")

Saved 50 podcasts to top_podcasts.json


In [5]:
#save episods of these podcasts to top_podcst.json

with open("top_podcasts.json", "r") as f:
    podcasts = json.load(f)

all_episodes = []

for podcast in podcasts:
    feed_id = podcast.get("id")
    podcast_title = podcast.get("title")

    print(f"Fetching episodes for: {podcast_title}")

    response = requests.get(
        "https://api.podcastindex.org/api/1.0/episodes/byfeedid",
        headers=headers,
        params= {"id": feed_id, "max": 1} #more episodes from one podcast - change max
    )

    if response.status_code == 200:
        episodes = response.json().get("items", [])
        for ep in episodes:
            all_episodes.append({
                "podcast": podcast_title,
                "episode_title": ep.get("title"),
                "audio_url": ep.get("enclosureUrl"),
                "id": ep.get("id")
            })
    

with open("top_episodes.json", "w") as f:
    json.dump(all_episodes, f, indent=4)

print(f"✅ Saved {len(all_episodes)} episodes to top_episodes.json")

Fetching episodes for: Kan Hourly News כאן רשת ב חדשות השעה
Fetching episodes for: Raw with Mindy and Boots
Fetching episodes for: Bulwark Takes
Fetching episodes for: Project 2025: The Ominous Specter
Fetching episodes for: Rated (R) Rucchi Show
Fetching episodes for: Locked On Orioles - Daily Podcast On The Baltimore Orioles
Fetching episodes for: Locked On Rangers - Daily Podcast On The Texas Rangers
Fetching episodes for: پیام محبت
Fetching episodes for: زندگی تازه
Fetching episodes for: آواز نجات
Fetching episodes for: فرزندان نور
Fetching episodes for: نان روزانۀ ما
Fetching episodes for: دورۀ شاگردسازی
Fetching episodes for: تمام کتاب
Fetching episodes for: Daily Motivational Positivity
Fetching episodes for: Beyond The Horizon
Fetching episodes for: Experiencia Kidzania Monterrey
Fetching episodes for: Locked On Red Sox - Daily Podcast On The Boston Red Sox
Fetching episodes for: On Life & Being Human
Fetching episodes for: Divine Providence: The Podcast
Fetching episodes for: 

[MODEL] Loading Whisper model: base on cpu
[THREADING] Using 4 threads
[FETCHING] Kan News @23 on 1/6/2025
[FETCHING] YouTube Channel Peak Retirement Planning 6/1/25
[FETCHING] The GOP’s Healthcare Flimflammery Is The New Big Lie
[FETCHING] Controversial Project 2025 Threatens to Centralize Executive Power and Erode Democratic Safeguards in the United States
[CONVERTING] Kan News @23 on 1/6/2025
[CONVERTING] YouTube Channel Peak Retirement Planning 6/1/25
[CONVERTING] Controversial Project 2025 Threatens to Centralize Executive Power and Erode Democratic Safeguards in the United States
[CONVERTING] The GOP’s Healthcare Flimflammery Is The New Big Lie
[TRANSCRIBING] Kan News @23 on 1/6/2025


/opt/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[TRANSCRIBING] YouTube Channel Peak Retirement Planning 6/1/25


/opt/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


[TRANSCRIBING] Controversial Project 2025 Threatens to Centralize Executive Power and Erode Democratic Safeguards in the United States


...

[TRANSCRIBING] _️ __INTRODUCING_ RATED R RUCCHI RADIO SHOW__ _ A_K_A_ Trench_ Princess    Get ready for the realest


/opt/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


[TRANSCRIBING] ORIOLES POSTCAST: Charlie Morton SHUTS DOWN Chicago White Sox, Baltimore Orioles earn sweep!


/opt/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


: 